<a href="https://colab.research.google.com/github/anu04596/DeepLearningTuotorial/blob/main/Day13(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Learning How CNN Model Works

Import Libraries

In [8]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import random
import numpy as np

Load Dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using", device)

Using cuda


In [3]:
tranform=transforms.ToTensor()
train_dataset=torchvision.datasets.FashionMNIST(root='./data',train=True,transform=tranform,download=True)
test_dataset=torchvision.datasets.FashionMNIST(root='./data',train=False,transform=tranform,download=True)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 203kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.78MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.12MB/s]


In [9]:
train_size = int(0.1 * len(train_dataset))  # 10% of 60,000 = 6,000
indices = np.random.choice(len(train_dataset), train_size, replace=False)
small_train_dataset = Subset(train_dataset, indices)

train_loader = DataLoader(small_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
print(len(train_loader))
print(len(test_loader))

94
157


CNN Model

In [13]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()

    self.conv1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,padding=1)
    self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=1)

    self.fc1=nn.Linear(in_features=32*7*7,out_features=128)
    self.fc2=nn.Linear(in_features=128,out_features=10)

  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,32*7*7)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x

Training

In [14]:
model=CNN().to(device)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
for epoch in range(5):
  total_loss=0
  for images, labels in train_loader:
    images, labels =images.to(device),labels.to(device)
    output=model(images)
    loss=criterion(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss+=loss.item()

  print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

Epoch 1, Loss: 0.9197483506608517
Epoch 2, Loss: 0.5264249634235463
Epoch 3, Loss: 0.4617704064288038
Epoch 4, Loss: 0.409966952940251
Epoch 5, Loss: 0.36367021786405684


Evaluate

In [16]:
correct=0
total=0
model.eval()
with torch.no_grad():
  for images, labels in test_loader:
    images, labels =images.to(device),labels.to(device)
    output=model(images)
    _,predicted=torch.max(output.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

print(f"Accuracy: {100*correct/total}%")

Accuracy: 84.16%
